In [1]:
from pathlib import Path
from pprint import pprint

import imageio
import torch

import lerobot
from lerobot.common.datasets.lerobot_dataset import LeRobotDataset 

/home/ns1254/miniforge3/envs/lerobot/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# repo_id = "lerobot/pusht" 
# dataset = LeRobotDataset(repo_id)

In [ ]:
import sys 
sys.path.append('/home/ns1254/lerobot/lerobot/common/datasets/push_dataset_to_hub/')

from aloha_hdf5_format import from_raw_to_lerobot_format

raw_dir = Path("/home/ns1254/act/dataset")
hf_dataset, episode_data_index, info = from_raw_to_lerobot_format(raw_dir, None, 20, False)
dataset=LeRobotDataset.from_preloaded(LeRobotDataset, hf_dataset=hf_dataset, episode_data_index=episode_data_index, info=info)

In [3]:
print(dataset)
print(dataset.hf_dataset)

LeRobotDataset(
  Repository ID: '<class 'lerobot.common.datasets.lerobot_dataset.LeRobotDataset'>',
  Split: 'train',
  Number of Samples: 20000,
  Number of Episodes: 50,
  Type: image (.png),
  Recorded Frames per Second: 20,
  Camera Keys: ['observation.images.top'],
  Video Frame Keys: N/A,
  Transformations: None,
  Codebase Version: v1.6,
)
Dataset({
    features: ['observation.images.top', 'observation.state', 'action', 'episode_index', 'frame_index', 'timestamp', 'next.done', 'index'],
    num_rows: 20000
})


In [4]:
print(f"\naverage number of frames per episode: {dataset.num_samples / dataset.num_episodes:.3f}")
print(f"frames per second used during data collection: {dataset.fps=}")
print(f"keys to access images from cameras: {dataset.camera_keys=}\n")


average number of frames per episode: 400.000
frames per second used during data collection: dataset.fps=20
keys to access images from cameras: dataset.camera_keys=['observation.images.top']



In [5]:
# save dataset to disk
dataset_path = Path("/home/ns1254/lerobot/dataset/custom")
# dataset.save(dataset_path)

In [9]:
# dir(dataset)
# dataset.save_to_disk(dataset_path)

In [11]:
# torch.save(dataset, dataset_path / "dataset.pt")

In [14]:
# ds= torch.load(dataset_path / "dataset.pt")
# ds

In [11]:
hf_dataset = dataset.hf_dataset.with_format(None)  # to remove transforms that cant be saved
hf_dataset.save_to_disk(dataset_path  )

Saving the dataset (2/2 shards): 100%|██████████| 20000/20000 [00:00<00:00, 28408.88 examples/s]


In [12]:
dataset

LeRobotDataset(
  Repository ID: '<class 'lerobot.common.datasets.lerobot_dataset.LeRobotDataset'>',
  Split: 'train',
  Number of Samples: 20000,
  Number of Episodes: 50,
  Type: image (.png),
  Recorded Frames per Second: 20,
  Camera Keys: ['observation.images.top'],
  Video Frame Keys: N/A,
  Transformations: None,
  Codebase Version: v1.6,
)

In [13]:
from lerobot.common.datasets.populate_dataset import *

In [15]:
# save_lerobot_dataset_on_disk(dataset)

In [ ]:
# stats = compute_stats(lerobot_dataset)

In [ ]:
lerobot_dataset=dataset
hf_dataset = lerobot_dataset.hf_dataset
info = lerobot_dataset.info
stats = lerobot_dataset.stats
episode_data_index = lerobot_dataset.episode_data_index
# local_dir = lerobot_dataset.videos_dir.parent
local_dir  = Path("/home/ns1254/lerobot/dataset/custom2")
meta_data_dir = local_dir / "meta_data"

hf_dataset = hf_dataset.with_format(None)  # to remove transforms that cant be saved
hf_dataset.save_to_disk(str(local_dir / "train"))

if stats==None: 
    stats={}
save_meta_data(info, stats, episode_data_index, meta_data_dir)

Saving the dataset (2/2 shards): 100%|██████████| 20000/20000 [00:00<00:00, 28445.59 examples/s]


In [6]:
# Access frame indexes associated to first episode
episode_index = 0
from_idx = dataset.episode_data_index["from"][episode_index].item()
to_idx = dataset.episode_data_index["to"][episode_index].item()

In [7]:
# LeRobot datasets actually subclass PyTorch datasets so you can do everything you know and love from working
# with the latter, like iterating through the dataset. Here we grab all the image frames.
frames = [dataset[idx]["observation.images.top"] for idx in range(from_idx, to_idx)]

# Video frames are now float32 in range [0,1] channel first (c,h,w) to follow pytorch convention. To visualize
# them, we convert to uint8 in range [0,255]
frames = [(frame * 255).type(torch.uint8) for frame in frames]
# and to channel last (h,w,c).
frames = [frame.permute((1, 2, 0)).numpy() for frame in frames]
len(frames)

400

In [8]:
# # Finally, we save the frames to a mp4 video for visualization.
Path("outputs/examples/1_load_lerobot_dataset").mkdir(parents=True, exist_ok=True)
imageio.mimsave("outputs/examples/1_load_lerobot_dataset/episode_0_custom_act.mp4", frames, fps=dataset.fps)

In [9]:
# For many machine learning applications we need to load the history of past observations or trajectories of
# future actions. Our datasets can load previous and future frames for each key/modality, using timestamps
# differences with the current loaded frame. For instance:
delta_timestamps = {
    # loads 4 images: 1 second before current frame, 500 ms before, 200 ms before, and current frame
    "observation.images.top": [-1, -0.5, -0.20, 0],
    # loads 8 state vectors: 1.5 seconds before, 1 second before, ... 20 ms, 10 ms, and current frame
    "observation.state": [-1.5, -1, -0.5, -0.20, -0.10, -0.02, -0.01, 0],
    # loads 64 action vectors: current frame, 1 frame in the future, 2 frames, ... 63 frames in the future
    "action": [t / dataset.fps for t in range(64)],
}
# dataset = LeRobotDataset(repo_id, delta_timestamps=delta_timestamps)
dataset=LeRobotDataset.from_preloaded(LeRobotDataset, hf_dataset=hf_dataset, episode_data_index=episode_data_index, info=info, delta_timestamps=delta_timestamps)

print(f"\n{dataset[0]['observation.images.top'].shape=}")  # (4,c,h,w)
print(f"{dataset[0]['observation.state'].shape=}")  # (8,c)
print(f"{dataset[0]['action'].shape=}\n")  # (64,c)


dataset[0]['observation.images.top'].shape=torch.Size([4, 3, 480, 640])
dataset[0]['observation.state'].shape=torch.Size([8, 14])
dataset[0]['action'].shape=torch.Size([64, 14])



In [10]:
# Finally, our datasets are fully compatible with PyTorch dataloaders and samplers because they are just
# PyTorch datasets.
dataloader = torch.utils.data.DataLoader(
    dataset,
    num_workers=0,
    batch_size=32,
    shuffle=True,
)

In [11]:
for batch in dataloader:
    print(f"{batch['observation.images.top'].shape=}")  # (32,4,c,h,w)
    print(f"{batch['observation.state'].shape=}")  # (32,8,c)
    print(f"{batch['action'].shape=}")  # (32,64,c)
    break


batch['observation.images.top'].shape=torch.Size([32, 4, 3, 480, 640])
batch['observation.state'].shape=torch.Size([32, 8, 14])
batch['action'].shape=torch.Size([32, 64, 14])


In [12]:
batch.keys()

dict_keys(['observation.images.top', 'observation.state', 'action', 'episode_index', 'frame_index', 'timestamp', 'next.done', 'index', 'observation.images.top_is_pad', 'observation.state_is_pad', 'action_is_pad'])

In [15]:
local_dir = "/home/ns1254/lerobot/dataset/act"
hf_dataset = hf_dataset.with_format(None)  # to remove transforms that cant be saved
hf_dataset.save_to_disk(local_dir)

Saving the dataset (2/2 shards): 100%|██████████| 20000/20000 [00:00<00:00, 28045.77 examples/s]
